In [1]:
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import shutil
import os

In [54]:
PWD = ''
IMGPATH = PWD + 'imgs\\'

In [55]:
images = os.listdir(IMGPATH + 'images')
labels = os.listdir(IMGPATH + 'labels')

In [56]:
trainImages, testImages, trainLabels, testLabels = train_test_split(
    images, labels, 
    test_size=0.2, random_state=69, shuffle=True
)

trainImages, valImages, trainLabels, valLabels = train_test_split(
    trainImages, trainLabels, test_size=0.25, random_state=69
) # 0.25 x 0.8 = 0.2

In [57]:
def MoveImages(imgs, labls, to='train'):
    os.mkdir(PWD + to)
    os.mkdir(PWD + to + '\\images')
    os.mkdir(PWD + to + '\\labels')

    NEWPATH = PWD + to

    print(f'Moving to {to}')

    for i, image in enumerate(imgs):
        shutil.move(IMGPATH + 'images\\' + image, NEWPATH + '\\images\\' + image)
        shutil.move(IMGPATH + 'labels\\' + labls[i], NEWPATH + '\\labels\\' + labls[i])
    

In [58]:
MoveImages(trainImages, trainLabels, to='train')
MoveImages(valImages, valLabels, to='valid')
MoveImages(testImages, testLabels, to='test')

Moving to train
Moving to valid
Moving to test


In [59]:
model = YOLO('yolov8n.pt')

In [60]:
model.train(data=PWD + 'data.yaml')

New https://pypi.org/project/ultralytics/8.3.21 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.102  Python-3.10.0 torch-2.4.1+cpu CPU (AMD Ryzen 5 3600 6-Core Processor)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:\Dev\Datasets\YoloCows\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

100%|███████████████████████████████████████████████████████████████████████████████| 755k/755k [00:00<00:00, 3.53MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning D:\Dev\Datasets\YoloCows\train\labels... 4869 images, 293 backgrounds, 418 corrupt: 100%|██████████| 48

train: WARNING  D:\Dev\Datasets\YoloCows\train\images\2024-05-16-12-55-04.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  D:\Dev\Datasets\YoloCows\train\images\2024-05-16-13-00-03.jpg: ignoring corrupt image/label: Label class 4 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  D:\Dev\Datasets\YoloCows\train\images\2024-05-16-13-05-03.jpg: ignoring corrupt image/label: Label class 4 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  D:\Dev\Datasets\YoloCows\train\images\2024-05-16-13-10-03.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  D:\Dev\Datasets\YoloCows\train\images\2024-05-16-13-20-03.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING  D:\Dev\Datasets\YoloCows\train\images\2024-05-16-13-40-03.jpg: i

...

C:\Users\Peblo\AppData\Roaming\Python\Python310\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.19 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning D:\Dev\Datasets\YoloCows\valid\labels... 1623 images, 115 backgrounds, 149 corrupt: 100%|██████████| 1623

val: WARNING  D:\Dev\Datasets\YoloCows\valid\images\2024-05-16-13-15-03.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  D:\Dev\Datasets\YoloCows\valid\images\2024-05-16-14-00-03.jpg: ignoring corrupt image/label: Label class 4 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  D:\Dev\Datasets\YoloCows\valid\images\2024-05-16-14-55-03.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  D:\Dev\Datasets\YoloCows\valid\images\2024-05-16-15-35-03.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  D:\Dev\Datasets\YoloCows\valid\images\2024-05-16-15-40-03.jpg: ignoring corrupt image/label: Label class 9 exceeds dataset class count 1. Possible class labels are 0-0
val: WARNING  D:\Dev\Datasets\YoloCows\valid\images\2024-05-16-16-00-03.jpg: ignoring corr

val: New cache created: D:\Dev\Datasets\YoloCows\valid\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 29, len(boxes) = 5835. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to runs\detect\train6\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train6
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      1.465      2.671      1.617        149        640:   1%|          | 3/279 [00:13<20:10,  4.


IndexError: index 1 is out of bounds for dimension 1 with size 1